In [ ]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display
import dblogin

In [ ]:
engine = create_engine("myswl+mysqldb://{}:{}@{}:{}/{}".format(dblogin.USER, dblogin.PASSWORD, dblogin.URI, dblogin.PORT, dblogin.DB), echo=True)

In [ ]:
sql = """
CREATE DATABASE IF NOT EXISTS dbikes;
"""
engine.execute(sql)

In [ ]:
for res in engine.execute("SHOW VARIABLES;"):
    print(res)

In [ ]:
sql = """
CREATE TABLES IF NOT EXISTS station (
address VARCHAR(256),
banking INTEGER,
bike_stands INTEGER,
bonus INTEGER,
contract_name VARCHAR(256),
name VARCHAR(256),
number INTEGER,
position_lat REAL,
position_lng REAL,
status VARCHAR(256)
)
"""
try:
    res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql)
    print(res.fetchall())
except: Exception as e:
    print(e)

In [ ]:
sql = """
CREATE TABLE IF NOT EXISTS availability(
number INTEGER,
available_bikes INTEGER,
available_bike_stands INTEGER,
last_update INTEGER
)
"""
try:
    res = engine.execute(sql)
    print(res.fetchall())
except: Exception as e:
    print(e) #traceback.format_exc())

In [ ]:
#!/usr/bon/env python
import requests 
import traceback
import datetime 
import time
APIKEY = "90823420398ff4"
NAME = "Dublin"
STATIONS = "https://api.jcdecaux.com/vls/vl/stations"

def write_to_file(text):
    with open("data/bikes_{}".format(now).replace(" ", "_"), "w") as f:
        f.write(r.text)

def write_to_db(text):
    pass


def main():
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS, params = {"apiKey" : APIKEY, "contract": NAME})
            print(r, now)
            write_to_file(r.text)
            write_to_db(r.text)
            time.sleep(5*60)
        
        except:
            print(traceback.format_exc())
            if engine is None:
    return

In [ ]:
def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations), len(stations))
    for station in stations:
        print(station)
        vals = (station.get('address'), int(station.get('banking')), station.get('bike_stands'), int(station.get('bonus')), station.get('contract_name'), station.get('name'), station.get('number'), station.get('position').get('lat'), station.get('position').get('lng'), station.get('status') )
        engine.execute("insert into station values(%s,%s,%s,%s,%s,%s,%s,%s, %s,%s)", vals)
        break
    return
stations_to_db(r.text)

In [ ]:
metadata = sqla.MetaData()
station = sqla.Table("station", metadata,
    sqla.Column('address', sql.String(256), nullable =False), sqla.Column('banking', sqla.Integer), sqla.Column('bike_stands', sqla.Integer), sqla.Column('bonus', sqla.Integer()), sqla.Column('contract_name', sqla.String(256)), sqla.Column('name', sqla.String(256)), sqla.Column('number', sqla.Integer), sqla.Column('position_lat', sqla.REAL), sqla.Column('position_lng', sqla.REAL), sqla.Column('status', sqla.String(256)))

availability = sqla.Table('availability', metadata,
    sqla.Column('available_bikes', sqla.Integer),sqla.Column('available_bike_stands', sqla.Integer),sqla.Column('number', sqla.Integer),sqla.Column('last_update', sqla.BigInteger))

try:
    station.drop(engine)
    availability.drop(engine)

except:
    pass

metadata.create_all(engine)

In [ ]:
#Use this code to load the tables if you have already created them
#metadata = sqla.MetaData(bind=engine)
#print(metadata)
#station = sqla.Table('station', metadata, autoload =True)
#print(station)

In [ ]:
#need to make sure the right keys are in the rows being inserted into the db
def stations_fix_keys(station):
    station['position_lat'] = station['position']['lat']
    station['position_lng'] = station['position']['lng']
    return station

stations =json.loads(open('stations.json', 'r').read())

engine.execute(station.insert(), *map(stations_fix_keys, stations))

In [ ]:
import pandas as pd
df = pd.read_sql_table("station", engine)

In [ ]:
display(df.head())

In [ ]:
sql = "select count(*) from station;"
#for row in engine.execute(sql):
#     print(row)
print(engine.execute(sql).fetchall())

In [ ]:
sql = "select name from station limit 10;"
for row in engine.execute(sql):
    print(row)

In [ ]:
#JOIN 
#(SELECT name, number from station)
#on AVAIL.number = station.number

sql = """
select * from
(select available_bike_stands, available_bikes, number from availability where available_bikes > 0)



"""

: 